In [129]:
import pandas as pd


df = pd.read_csv("data/properties.csv")
geo = pd.read_csv("data/postal-codes-belgium.csv", sep=";")[['Postal Code', 'Région code', 'Province code']]  # geo file: postal - province?

# drop columns
df = df.drop(['id', 'region', 'locality', 'zip_code', 'latitude', 'longitude', 'subproperty_type', 'construction_year', 'nbr_frontages', 'primary_energy_consumption_sqm', 'cadastral_income'], axis='columns')
df = df[df['nbr_bedrooms']< 6]
df = df.drop('nbr_bedrooms', axis='columns')
# examine features: rows, unique values, count empty/'MISSING' values
all_counts = {}
for i in df.columns:
    unique = len(df[i].unique())
    empty_count = df[i].isnull().sum()
    missing_count = 0
    if df[i].apply(lambda x: isinstance(x, str)).all():
        missing_count = df[i].str.contains('MISSING').sum()
    all_counts[i] = {'unique': unique, 'empty': empty_count, 'missing': missing_count}
bad_values = pd.DataFrame(all_counts).T
display(bad_values)

# check for duplicates
if not df.duplicated().any():
    print('No duplicates')
else:
    df.drop_duplicates()
    print('duplicates removed')


#print(df.fl_garden.value_counts())

print(df.shape)
# remove province 'MISSING' (3 rows)
df = df[~df['province'].str.contains('MISSING')]
print(df.shape)
df = df[~df['equipped_kitchen'].str.contains('MISSING')]
print(df.shape)
#remove total area = empty
df = df[~df['total_area_sqm'].isnull()]
print(df.shape)
#remove surface land = empty for houses
df = df[df['surface_land_sqm'].notnull() | df['property_type'] != 'HOUSE']
print(df.shape)
#terrace: remove empty, remove 0 sqm when fl is 1
df = df[(df['fl_terrace'] == 0) | (df['fl_terrace'] == 1 & df['terrace_sqm'].notnull())]
print(df.shape)
#garden: remove empty, remove 0 sqm when fl is 1
df = df[(df['fl_garden'] == 0) | (df['fl_garden'] == 1 & df['garden_sqm'].notnull())]
print(df.shape)
# remove price > 1mil
df = df[df['price'] < 1000000]
print(df.shape)
# remove building state 'MISSING'
df = df[df['state_building'] != 'MISSING']
print(df.shape)
# remove epc 'MISSING'
df = df[df['epc'] != 'MISSING']
print(df.shape)
# remove heating_type 'MISSING'
df = df[df['heating_type'] != 'MISSING']
print(df.shape)

# check binary change garden/terrace: print?

# examine features after cleaning
all_counts = {}
for i in df.columns:
    unique = len(df[i].unique())
    empty_count = df[i].isnull().sum()
    missing_count = 0
    if df[i].apply(lambda x: isinstance(x, str)).all():
        missing_count = df[i].str.contains('MISSING').sum()
    all_counts[i] = {'unique': unique, 'empty': empty_count, 'missing': missing_count}
bad_values = pd.DataFrame(all_counts).T
display(bad_values)

# detect outliers: boxplots?


,unique,empty,missing
price,8175,0,0
property_type,2,0,0
province,12,0,3
total_area_sqm,796,7223,0
surface_land_sqm,3441,36157,0
equipped_kitchen,9,0,31824
fl_furnished,2,0,0
fl_open_fire,2,0,0
fl_terrace,2,0,0
terrace_sqm,227,12370,0


duplicates removed
(72645, 18)
(72642, 18)
(40821, 18)
(38221, 18)
(38221, 18)
(31507, 18)
(30131, 18)
(29220, 18)
(20113, 18)
(15231, 18)
(12595, 18)


,unique,empty,missing
price,982,0,0
property_type,2,0,0
province,11,0,0
total_area_sqm,463,0,0
surface_land_sqm,1614,6042,0
equipped_kitchen,8,0,0
fl_furnished,2,0,0
fl_open_fire,2,0,0
fl_terrace,2,0,0
terrace_sqm,156,0,0


In [ ]:

from sklearn.preprocessing import OrdinalEncoder, OneHotEncoder, MinMaxScaler
from sklearn.model_selection import train_test_split
from sklearn.linear_model import LinearRegression
import seaborn as sn
from matplotlib import pyplot as plt
import numpy as np

# encoding categorical variables (or try labler?)
ordinals = {'state_building': [['AS_NEW', 'JUST_RENOVATED', 'GOOD', 'TO_BE_DONE_UP', 'TO_RENOVATE', 'TO_RESTORE']], 
           'province': [['West Flanders', 'East Flanders', 'Walloon Brabant', 'Brussels', 'Hainaut', 'Antwerp', 'Liège', 'Namur', 'Flemish Brabant', 'Limburg', 'Luxembourg']], 
           'equipped_kitchen': [['NOT_INSTALLED', 'USA_UNINSTALLED', 'INSTALLED', 'USA_INSTALLED', 'SEMI_EQUIPPED', 'USA_SEMI_EQUIPPED', 'HYPER_EQUIPPED', 'USA_HYPER_EQUIPPED']], 
           'epc': [['A++', 'A+', 'A', 'B', 'C', 'D', 'E', 'F', 'G']], 
           'heating_type': [['SOLAR', 'ELECTRIC', 'GAS', 'PELLET', 'WOOD', 'FUELOIL', 'CARBON']]}
for i in ordinals:
    df[i] = OrdinalEncoder(categories=ordinals[i], dtype=int).fit_transform(df[[i]].to_numpy()) + 1

# normalize float variables
fl = (df.drop('price', axis=1).dtypes == 'float')
fl_cols = list(fl[fl].index)
for i in fl_cols:
    df[i] = MinMaxScaler().fit_transform(df[[i]])

# create seperate datasets: APARTMENT / HOUSE
df_house = df[df['property_type'] == 'HOUSE'].drop(columns='property_type')
df_apartment = df[df['property_type'] == 'APARTMENT'].drop(columns=['property_type', 'surface_land_sqm'])

# split datasets
X_house, y_house = np.array(df_house.drop(columns='price')), np.array(df_house['price'])
X_house_train, X_house_test, y_house_train, y_house_test = train_test_split(X_house, y_house, random_state=42, test_size=0.2)

X_apartment, y_apartment = np.array(df_apartment.drop(columns='price')), np.array(df_apartment['price'])
X_apartment_train, X_apartment_test, y_apartment_train, y_apartment_test = train_test_split(X_apartment, y_apartment, random_state=42, test_size=0.2)

enc = LinearRegression()
enc.fit(X_apartment_train, y_apartment_train)
print(enc.score(X_apartment_train, y_apartment_train))
enc.fit(X_house_train, y_house_train)
print(enc.score(X_house_train, y_house_train))




0.5253888392665866
0.39913397713086973
